In [1]:
# importacoes
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import openpyxl
import time
import pandas as pd

In [2]:
# definindo lista de empresas:
lista_br = pd.read_excel('lista_hospitais_na_bolsa.xlsx')
tickers = lista_br['id_acao']
tickers

0    AALR3
1    DASA3
2    FLRY3
3    HAPV3
4    KRSA3
5    MATD3
6    ODPV3
7    ONCO3
8    QUAL3
9    RDOR3
Name: id_acao, dtype: object

In [3]:
# - 1 utilização do webdriver

browser = webdriver.Chrome()
time.sleep(2)
browser.get('https://statusinvest.com.br/acoes/')
time.sleep(15)


In [4]:
# remover propagandas
ad_close = browser.find_element(By.XPATH, '//button[@class="btn-close"]')
ad_close.click()
time.sleep(1)

In [5]:
# raspar os históricos de RESULTADO
with pd.ExcelWriter('raspagem_tabela_resultad-.xlsx', engine='openpyxl') as writer:
    for ticker in tickers:
        #Acessando conteúdo de cada empresa
        lupa = browser.find_element(By.XPATH, '//a[@class="main-search" and @title="Busca por nome de empresas, FIIs, índices e muito mais"]')
        lupa.click()
        input =  browser.find_element(By.XPATH, '//input[@class="Typeahead-input input tt-input"]')
        input.send_keys(ticker)
        time.sleep(3)
        input.send_keys(Keys.ARROW_DOWN)
        input.send_keys(Keys.ENTER)
        time.sleep(3)
        
        # SCROLLS para carregar página
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # clicar em TODOS ver mais para html carregar
        #detecta quantos ver mais tem:
        ver_mais = browser.find_elements(By.XPATH, '//button[@title="Ver tudo"]')
        len(ver_mais) # tem 3 ou 4 'ver mais'
        time.sleep(1)

        # gera loop para clicar em todos botoes 'ver mais':
        for botao in range(len(ver_mais)):
            # Rolar até o ver mais
            browser.execute_script("arguments[0].scrollIntoView(true);", ver_mais[botao])
            browser.execute_script("window.scrollBy(0, -400);")
            time.sleep(1)

            #clicar no ver mais
            ver_mais[botao].click()
            time.sleep(2)
        # simplificar tabelas (eliminar análises horizontais)
        # encontrar input que quero clicar
        # tem um botao verde para cada uma das 3 tabelas
        botao_verde = browser.find_elements(By.XPATH, '//div[@data-tooltip="Apresentar/esconder colunas do grid"]')

        for botao in range(len(botao_verde)):
            browser.execute_script("arguments[0].scrollIntoView(true);", botao_verde[botao])
            time.sleep(1)
            browser.execute_script("window.scrollBy(0, -300);")
            time.sleep(1)
            input_bv1 = botao_verde[botao].find_element(By.TAG_NAME, 'input')
            input_bv1.click()
            #encontrar checkbox e clicar
            ul_bv = botao_verde[botao].find_element(By.TAG_NAME, 'ul')
            li = ul_bv.find_elements(By.TAG_NAME, 'li')
            for j in range(len(li)):
                if (li[j].text == 'AH'):
                    checkbox = li[j]
                    checkbox.click()
                    time.sleep(2)
            checkbox.send_keys(Keys.ESCAPE)
                    
        # divs resultado fluxo e balanço
        divs_resultado_e_fluxo = browser.find_elements(By.XPATH, '//div[@class="DRE table-info card card-panel white mb-5"]')
        div_resultado = divs_resultado_e_fluxo[0]
        div_resultado_inteira_text = div_resultado.text.split('\n') 
        print(len(div_resultado_inteira_text)) # guardar - variável com textos da div com resultados
        div_fluxo = divs_resultado_e_fluxo[1]
        div_fluxo_inteira_text = div_fluxo.text.split('\n')
        print(len(div_fluxo_inteira_text)) # guardar - variável com textos da div com fluxo
        time.sleep(2)
        
        # encontrando o número de anos
        # divs_th = browser.find_elements(By.XPATH, '//th[@class="text-right DATA timeType-0"]') # o total de divs th com essa classe estão divididos entre as três tabelas com valores financeiros (resultado, fluxo de caixa e balanço patrimial)
        # numero_de_anos = len(divs_th)/3
        # numero_de_anos = int(numero_de_anos)
        # anos = []
        # for ano in range(numero_de_anos):
        #     anos.append(divs_th[ano].text) 
        
        divs_th = browser.find_elements(By.XPATH, '//th[@class="text-right DATA timeType-0"]') # o total de divs th com essa classe estão divididos entre as três tabelas com valores financeiros (resultado, fluxo de caixa e balanço patrimial)
        numero_de_anos = len(divs_th)/3
        numero_de_anos = int(numero_de_anos)
        anos = []
        for ano in range(len(divs_th)):
            #print(divs_th[ano].text)
            anos.append(divs_th[ano].text)
        print(anos)
            
        # Criar um conjunto para armazenar anos únicos
        anos_passados = set()

        # Lista para armazenar anos não repetidos
        anos_nao_repetidos = []

        # Percorrer a lista de anos
        for ano in anos:
        # Verificar se o ano já foi visitado
            if ano not in anos_passados:
                anos_nao_repetidos.append(ano)
                anos_passados.add(ano)
            else:
                # Se encontrar um ano repetido, parar o loop
                break

        # Imprimir a lista de anos não repetidos
        print(anos_nao_repetidos) # printa a array com os anos que irão compor o header do dataframe
        numero_de_anos = len(anos_nao_repetidos)
        # print(anos) 
        
        # pegando valores conforme padrão detectado no listão de textos:
        # Receita Líquida
        receita_liquida = []
        indice_inicial = 15
        
        for texto in range(15, 15 + numero_de_anos):
            receita_liquida.append(div_resultado_inteira_text[texto])
        print(receita_liquida)
        
        
        ##################################################
        custos = []
        for texto in range(15 + numero_de_anos + 3, 15 + numero_de_anos + 3 + numero_de_anos):
            custos.append(div_resultado_inteira_text[texto])
        print(custos)
        
        #time.sleep(2)
        
        lucro_bruto = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos):
            lucro_bruto.append(div_resultado_inteira_text[texto])
        print(lucro_bruto)
        
       # time.sleep(2)
        
        despesas_receitas_operacionais = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos):
            despesas_receitas_operacionais.append(div_resultado_inteira_text[texto])
        print(despesas_receitas_operacionais)
        
        #time.sleep(2)
        
        ebtida = []
        for texto in range( 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos):
            ebtida.append(div_resultado_inteira_text[texto])
        print(ebtida)
            
        #time.sleep(2)
        
        amort_deprec = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            amort_deprec.append(div_resultado_inteira_text[texto])
        print(amort_deprec)

        ebit = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            ebit.append(div_resultado_inteira_text[texto])
        print(ebit)
        
        #time.sleep(2)
        
        res_n_oper = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            res_n_oper.append(div_resultado_inteira_text[texto])
        print(res_n_oper)
        
       #time.sleep(2)
        
        resultado_financeiro = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            resultado_financeiro.append(div_resultado_inteira_text[texto])
        print(resultado_financeiro)
          
        #time.sleep(2)  
        
        impostos = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos):
            impostos.append(div_resultado_inteira_text[texto])
        print(impostos)
        
        #time.sleep(2)
        
        lucro_liquido = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos):
            lucro_liquido.append(div_resultado_inteira_text[texto])
        print(lucro_liquido)
         
        #time.sleep(2)   
        
        lucro_controladora = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos):
            lucro_controladora.append(div_resultado_inteira_text[texto])
        print(lucro_controladora)
        
        #time.sleep(2) 
           
        lucro_nao_controladores = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos):
            lucro_nao_controladores.append(div_resultado_inteira_text[texto])
        print(lucro_nao_controladores)
        
        capex = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos):
            capex.append(div_resultado_inteira_text[texto])
        print(capex)
         
        #time.sleep(2)   
        
        divida_bruta = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos): 
            divida_bruta.append(div_resultado_inteira_text[texto])
        print(divida_bruta)
        
        #time.sleep(2)   
    
         
        divida_liquida = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            divida_liquida.append(div_resultado_inteira_text[texto])
        print(divida_liquida)
        
        #time.sleep(2)
        
        roe = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            roe.append(div_resultado_inteira_text[texto])
        print(roe)
        
        #time.sleep(2)
        
        roic = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            roic.append(div_resultado_inteira_text[texto])
        print(roic)
         
        margem_bruta = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            margem_bruta.append(div_resultado_inteira_text[texto])
        print(margem_bruta)

        margem_ebtida = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos):
            margem_ebtida.append(div_resultado_inteira_text[texto])
        print(margem_ebtida)

        margem_liquida = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos):
            margem_liquida.append(div_resultado_inteira_text[texto])
        print(margem_liquida)

        div_liq = []
        for texto in range(15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4, 15 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos + 3 + numero_de_anos + 4 + numero_de_anos + 4 + numero_de_anos):
            div_liq.append(div_resultado_inteira_text[texto])
        print(div_liq)
            
        raspagem = {
            'Ano': anos_nao_repetidos,
            'Receita Líquida - (R$)': receita_liquida,
            'Custos - (R$)':custos,
            'Lucro Bruto - (R$)': lucro_bruto,
            'Despesas/Receitas Operacionais - (R$)': despesas_receitas_operacionais,
            'EBITDA - (R$)': ebtida,
            'Amortização/Depreciação': amort_deprec,
            'EBIT - (R$)': ebit,
            'Resultado não operacional - (R$)':res_n_oper,
            'Resultado Financeiro - (R$)':resultado_financeiro,
            'Impostos - (R$)':impostos,
            'Lucro Líquido - (R$)':lucro_liquido,
            'Lucro atribuído a Controladora':lucro_controladora,
            'Lucro atribuído a Não Controladores':lucro_nao_controladores,
            'CAPEX - (R$)':capex,
            'Dívida Bruta - (R$)':divida_bruta,
            'Dívida Líquida - (R$)':divida_liquida,
            'ROE - (%)':roe,
            'ROIC - (%)':roic,
            'Margem Bruta - (%)':margem_bruta,
            'Margem Ebitda - (%)':margem_ebtida,
            'Margem Líquida - (%)':margem_liquida,
            'Dívida Líquida/Ebitda':div_liq,
        }


        # NOME DA EMPRESA PARA PÔR EM CADA LINHA
        company_description = browser.find_element(By.XPATH, '//div[@class="company-description w-100 w-md-70 ml-md-5"]')
        nome_cnpj = company_description.text.split('\n')
        nome_empresa = nome_cnpj[0]
        nome_empresa = nome_empresa.replace('/', '.')
        cnpj_empresa = nome_cnpj[1]
        
        # ADICIONANDO NOME E CNPJ AO DATAFRAME
        
        raspagem_df = pd.DataFrame(raspagem)

        numero_linhas = raspagem_df.shape[0]
        # numero_linhas

        empresa = []
        for n in range(numero_linhas):
            empresa.append(nome_empresa)
        # empresa
        raspagem_df.insert(0, 'empresa', empresa)

        cnpj = []
        for n in range(numero_linhas):
            cnpj.append(cnpj_empresa)
        # cnpj

        raspagem_df.insert(1, 'cnpj', cnpj)
        # raspagem_df

        raspagem_df.to_excel(writer, sheet_name=nome_empresa[:18], header=True, index=False)
        display(raspagem_df)

206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['1.085,01 M', '1.136,57 M', '928,79 M', '1.072,87 M', '1.076,92 M']
['-759,26 M', '-780,53 M', '-763,47 M', '-788,05 M', '-787,54 M']
['325,75 M', '356,04 M', '165,32 M', '284,82 M', '289,38 M']
['-357,79 M', '-267,87 M', '-174,45 M', '-142,16 M', '-152,09 M']
['85,70 M', '207,78 M', '110,05 M', '260,53 M', '219,91 M']
['-117,74 M', '-119,60 M', '-119,18 M', '-117,87 M', '-82,62 M']
['-32,04 M', '88,17 M', '-9,12 M', '142,66 M', '137,29 M']
['-', '-', '-', '0,00', '0,00']
['-201,20 M', '-85,72 M', '-63,94 M', '-76,95 M', '-67,24 M']
['13,99 M', '-0,47 M', '-19,36 M', '-15,64 M', '-18,48 M']
['-219,24 M', '1,99 M', '-92,43 M', '50,07 M', '51,57 M']
['-227,81 M', '-5,63 M', '-97,42 M', '41,30 M', '40,09 M']
['8,57 M', '7,61 M', '4,99 M', '8,77 M', '11,48 M']
['-', '-', '-', '-', '-']
['-', '777,99 M', '793,84 M', '614,7

,empresa,cnpj,Ano,Receita Líquida - (R$),Custos - (R$),Lucro Bruto - (R$),Despesas/Receitas Operacionais - (R$),EBITDA - (R$),Amortização/Depreciação,EBIT - (R$),...,Lucro atribuído a Não Controladores,CAPEX - (R$),Dívida Bruta - (R$),Dívida Líquida - (R$),ROE - (%),ROIC - (%),Margem Bruta - (%),Margem Ebitda - (%),Margem Líquida - (%),Dívida Líquida/Ebitda
0,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2022,"1.085,01 M","-759,26 M","325,75 M","-357,79 M","85,70 M","-117,74 M","-32,04 M",...,"8,57 M",-,-,-,"-22,33%","-2,28%","30,02%","7,90%","-20,21%",-
1,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2021,"1.136,57 M","-780,53 M","356,04 M","-267,87 M","207,78 M","-119,60 M","88,17 M",...,"7,61 M",-,"777,99 M",-,"0,16%","4,41%","31,33%","18,28%","0,17%",-
2,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2020,"928,79 M","-763,47 M","165,32 M","-174,45 M","110,05 M","-119,18 M","-9,12 M",...,"4,99 M",-,"793,84 M",-,"-7,55%","-1,41%","17,80%","11,85%","-9,95%",-
3,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2019,"1.072,87 M","-788,05 M","284,82 M","-142,16 M","260,53 M","-117,87 M","142,66 M",...,"8,77 M",-,"614,77 M",-,"3,79%","6,57%","26,55%","24,28%","4,67%",-
4,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2018,"1.076,92 M","-787,54 M","289,38 M","-152,09 M","219,91 M","-82,62 M","137,29 M",...,"11,48 M",-,"605,51 M",-,"3,97%","6,24%","26,87%","20,42%","4,79%",-


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['13.128,89 M', '10.418,67 M', '7.039,33 M', '4.364,59 M', '3.933,45 M']
['-9.330,45 M', '-7.301,85 M', '-5.283,62 M', '-2.818,18 M', '-2.536,68 M']
['3.798,45 M', '3.116,82 M', '1.755,71 M', '1.546,41 M', '1.396,77 M']
['-2.781,85 M', '-3.028,31 M', '-1.589,16 M', '-1.148,13 M', '-1.051,08 M']
['2.128,12 M', '1.018,28 M', '932,14 M', '853,93 M', '605,49 M']
['-1.111,52 M', '-929,76 M', '-765,59 M', '-455,65 M', '-259,80 M']
['1.016,60 M', '88,52 M', '166,55 M', '398,28 M', '345,69 M']
['-', '-', '-', '0,00', '0,00']
['-1.595,39 M', '-462,74 M', '-385,99 M', '-259,66 M', '-151,66 M']
['191,03 M', '157,29 M', '71,70 M', '-8,40 M', '-8,73 M']
['-387,77 M', '-216,94 M', '-147,74 M', '130,22 M', '185,31 M']
['-386,92 M', '-226,57 M', '-150,80 M', '124,78 M', '184,38 M']
['-0,84 M', '9,63 M', '3,06 M', '5,45 M', '0,93 M']
[

,empresa,cnpj,Ano,Receita Líquida - (R$),Custos - (R$),Lucro Bruto - (R$),Despesas/Receitas Operacionais - (R$),EBITDA - (R$),Amortização/Depreciação,EBIT - (R$),...,Lucro atribuído a Não Controladores,CAPEX - (R$),Dívida Bruta - (R$),Dívida Líquida - (R$),ROE - (%),ROIC - (%),Margem Bruta - (%),Margem Ebitda - (%),Margem Líquida - (%),Dívida Líquida/Ebitda
0,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2022,"13.128,89 M","-9.330,45 M","3.798,45 M","-2.781,85 M","2.128,12 M","-1.111,52 M","1.016,60 M",...,"-0,84 M",-,-,-,"-5,47%","4,45%","28,93%","16,21%","-2,95%",-
1,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2021,"10.418,67 M","-7.301,85 M","3.116,82 M","-3.028,31 M","1.018,28 M","-929,76 M","88,52 M",...,"9,63 M",-,"7.646,85 M",-,"-2,87%","-0,45%","29,92%","9,77%","-2,08%",-
2,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2020,"7.039,33 M","-5.283,62 M","1.755,71 M","-1.589,16 M","932,14 M","-765,59 M","166,55 M",...,"3,06 M",-,"5.338,82 M",-,"-4,18%","1,07%","24,94%","13,24%","-2,10%",-
3,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2019,"4.364,59 M","-2.818,18 M","1.546,41 M","-1.148,13 M","853,93 M","-455,65 M","398,28 M",...,"5,45 M",-,"2.824,80 M",-,"3,99%","6,40%","35,43%","19,56%","2,98%",-
4,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2018,"3.933,45 M","-2.536,68 M","1.396,77 M","-1.051,08 M","605,49 M","-259,80 M","345,69 M",...,"0,93 M",-,"2.543,28 M",-,"5,58%","5,75%","35,51%","15,39%","4,71%",-


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['4.463,04 M', '3.872,65 M', '2.971,62 M', '2.905,11 M', '2.664,46 M']
['-3.252,93 M', '-2.745,76 M', '-2.161,89 M', '-2.016,72 M', '-1.855,96 M']
['1.210,11 M', '1.126,89 M', '809,73 M', '888,39 M', '808,50 M']
['-475,14 M', '-459,63 M', '-331,06 M', '-328,21 M', '-305,25 M']
['1.189,71 M', '1.055,99 M', '837,46 M', '880,98 M', '-']
['-455,34 M', '-389,28 M', '-358,47 M', '-320,46 M', '-']
['734,97 M', '667,26 M', '478,67 M', '560,18 M', '503,25 M']
['-', '-', '-', '0,00', '0,00']
['-329,58 M', '-171,15 M', '-137,94 M', '-133,98 M', '-52,11 M']
['-96,66 M', '-144,62 M', '-83,77 M', '-113,88 M', '-119,56 M']
['308,73 M', '351,49 M', '256,96 M', '312,32 M', '331,59 M']
['307,91 M', '349,93 M', '256,96 M', '312,32 M', '331,59 M']
['0,82 M', '1,57 M', '0,00', '0,00', '0,00']
['-', '-', '-', '-', '-']
['3.678,09 M', '2.948

...

,empresa,cnpj,Ano,Receita Líquida - (R$),Custos - (R$),Lucro Bruto - (R$),Despesas/Receitas Operacionais - (R$),EBITDA - (R$),Amortização/Depreciação,EBIT - (R$),...,Lucro atribuído a Não Controladores,CAPEX - (R$),Dívida Bruta - (R$),Dívida Líquida - (R$),ROE - (%),ROIC - (%),Margem Bruta - (%),Margem Ebitda - (%),Margem Líquida - (%),Dívida Líquida/Ebitda
0,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2022,"4.088,25 M","-2.636,15 M","1.452,11 M","-981,54 M","646,25 M","-175,15 M","470,57 M",...,"67,51 M",-,-,-,"4,32%","7,58%","35,52%","15,81%","2,78%",-
1,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2021,"2.702,07 M","-1.833,28 M","868,79 M","-691,58 M","304,19 M","-126,97 M","177,22 M",...,"24,48 M",-,"2.430,37 M",-,"-0,97%","1,95%","32,15%","11,26%","-0,83%",-
2,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2020,"2.035,19 M","-1.399,51 M","635,68 M","-616,06 M","126,43 M","-106,81 M","19,62 M",...,"36,35 M",-,"1.632,33 M",-,"-17,81%","-2,10%","31,23%","6,21%","-5,96%",-


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['1.951,41 M', '2.096,55 M', '2.025,95 M', '2.003,01 M', '1.932,12 M']
['-444,22 M', '-425,87 M', '-372,92 M', '-379,48 M', '-368,62 M']
['1.507,19 M', '1.670,68 M', '1.653,04 M', '1.623,53 M', '1.563,50 M']
['-1.144,90 M', '-1.033,89 M', '-1.043,81 M', '-1.018,95 M', '-907,35 M']
['782,60 M', '1.002,27 M', '976,71 M', '985,32 M', '-']
['-415,42 M', '-362,84 M', '-367,48 M', '-380,74 M', '-']
['362,29 M', '636,79 M', '609,23 M', '604,57 M', '656,15 M']
['-', '-', '-', '0,00', '0,00']
['-215,56 M', '-82,44 M', '-56,94 M', '-36,75 M', '-28,04 M']
['-45,25 M', '-177,05 M', '-157,10 M', '-175,07 M', '-231,07 M']
['101,48 M', '377,31 M', '395,19 M', '392,76 M', '397,04 M']
['92,82 M', '365,81 M', '392,10 M', '392,76 M', '397,04 M']
['8,66 M', '11,49 M', '3,09 M', '0,00', '0,00']
['-', '-', '-', '-', '-']
['2.257,71 M', '2.2

,empresa,cnpj,Ano,Receita Líquida - (R$),Custos - (R$),Lucro Bruto - (R$),Despesas/Receitas Operacionais - (R$),EBITDA - (R$),Amortização/Depreciação,EBIT - (R$),...,Lucro atribuído a Não Controladores,CAPEX - (R$),Dívida Bruta - (R$),Dívida Líquida - (R$),ROE - (%),ROIC - (%),Margem Bruta - (%),Margem Ebitda - (%),Margem Líquida - (%),Dívida Líquida/Ebitda
0,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2022,"1.951,41 M","-444,22 M","1.507,19 M","-1.144,90 M","782,60 M","-415,42 M","362,29 M",...,"8,66 M",-,"2.257,71 M",-,"7,48%","8,77%","77,24%","40,10%","5,20%",-
1,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2021,"2.096,55 M","-425,87 M","1.670,68 M","-1.033,89 M","1.002,27 M","-362,84 M","636,79 M",...,"11,49 M",-,"2.226,21 M",-,"29,59%","13,13%","79,69%","47,81%","18,00%",-
2,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2020,"2.025,95 M","-372,92 M","1.653,04 M","-1.043,81 M","976,71 M","-367,48 M","609,23 M",...,"3,09 M",-,"1.351,00 M",-,"23,45%","14,89%","81,59%","48,21%","19,51%",-
3,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2019,"2.003,01 M","-379,48 M","1.623,53 M","-1.018,95 M","985,32 M","-380,74 M","604,57 M",...,"0,00",-,"1.330,40 M",-,"27,53%","15,58%","81,05%","49,19%","19,61%",-
4,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2018,"1.932,12 M","-368,62 M","1.563,50 M","-907,35 M",-,-,"656,15 M",...,"0,00",-,"615,73 M",-,"16,86%","14,31%","80,92%",-,"20,55%",-


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['22.987,44 M', '20.381,88 M', '14.029,41 M', '13.319,84 M', '10.914,89 M']
['-17.785,66 M', '-15.572,11 M', '-11.462,35 M', '-9.838,96 M', '-8.109,53 M']
['5.201,78 M', '4.809,77 M', '2.567,06 M', '3.480,87 M', '2.805,36 M']
['-1.302,94 M', '-1.141,40 M', '-929,83 M', '-705,02 M', '-528,38 M']
['5.302,98 M', '4.897,04 M', '2.481,75 M', '3.484,65 M', '2.670,25 M']
['-1.404,14 M', '-1.228,67 M', '-844,53 M', '-708,80 M', '-393,27 M']
['3.898,84 M', '3.668,37 M', '1.637,23 M', '2.775,85 M', '2.276,98 M']
['-', '-', '-', '-', '-']
['-2.573,58 M', '-1.635,75 M', '-1.153,77 M', '-1.076,90 M', '-631,60 M']
['-62,95 M', '-354,88 M', '-24,03 M', '-507,44 M', '-465,77 M']
['1.262,32 M', '1.677,74 M', '459,42 M', '1.191,52 M', '1.179,62 M']
['1.194,97 M', '1.575,31 M', '428,33 M', '1.167,07 M', '1.169,96 M']
['67,35 M', '102,43 

,empresa,cnpj,Ano,Receita Líquida - (R$),Custos - (R$),Lucro Bruto - (R$),Despesas/Receitas Operacionais - (R$),EBITDA - (R$),Amortização/Depreciação,EBIT - (R$),...,Lucro atribuído a Não Controladores,CAPEX - (R$),Dívida Bruta - (R$),Dívida Líquida - (R$),ROE - (%),ROIC - (%),Margem Bruta - (%),Margem Ebitda - (%),Margem Líquida - (%),Dívida Líquida/Ebitda
0,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2022,"22.987,44 M","-17.785,66 M","5.201,78 M","-1.302,94 M","5.302,98 M","-1.404,14 M","3.898,84 M",...,"67,35 M",-,-,-,"5,47%","6,85%","22,63%","23,07%","5,49%",-
1,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2021,"20.381,88 M","-15.572,11 M","4.809,77 M","-1.141,40 M","4.897,04 M","-1.228,67 M","3.668,37 M",...,"102,43 M",-,"27.629,34 M",-,"11,43%","7,83%","23,60%","24,03%","8,23%",-
2,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2020,"14.029,41 M","-11.462,35 M","2.567,06 M","-929,83 M","2.481,75 M","-844,53 M","1.637,23 M",...,"31,09 M",-,"23.938,54 M",-,"3,28%","4,25%","18,30%","17,69%","3,27%",-
3,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2019,"13.319,84 M","-9.838,96 M","3.480,87 M","-705,02 M","3.484,65 M","-708,80 M","2.775,85 M",...,"24,45 M",-,"15.406,38 M",-,"21,89%","10,88%","26,13%","26,16%","8,95%",-
4,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2018,"10.914,89 M","-8.109,53 M","2.805,36 M","-528,38 M","2.670,25 M","-393,27 M","2.276,98 M",...,"9,65 M",-,"11.846,59 M",-,"26,05%","11,06%","25,70%","24,46%","10,81%",-
